In [2]:
!pip install git+https://github.com/huggingface/transformers -U
!pip install accelerate
!pip install -i https://pypi.org/simple/ bitsandbytes

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-ifnl5_iz
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-ifnl5_iz
  Resolved https://github.com/huggingface/transformers to commit 965cf677695dd363285831afca8cf479cf0c600c
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.39.0.dev0-py3-none-any.whl size=8662804 sha256=22971fcc58f14b00f8347774542f673e88a62037875f1a531c3eae026eff8a14
  Stored in directory: /tmp/pip-ephem-wheel-cache-j1yn5bbs/wheels/c0/14/d6/6c9a5582d2ac191ec0a483be151a4495fe1eb2a6706ca49f1b
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.38.1
    Uninstalling transformers-4.38.1:
      Successfully uninstalled transformers-4.38.1
Looking in indexes: https://pypi.org/sim

In [3]:
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch 


access_token_read = UserSecretsClient().get_secret("HUGGINGFACE_ACCESS_TOKEN")
login(token = access_token_read)

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [4]:
quantization_config = BitsAndBytesConfig(load_in_4bit=True)
tokenizer = AutoTokenizer.from_pretrained("google/gemma-7b")
model = AutoModelForCausalLM.from_pretrained(
    "google/gemma-7b", 
    low_cpu_mem_usage=True, 
    quantization_config=quantization_config, 
    device_map="auto"
)#.to(device)

tokenizer_config.json:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/555 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/2.11G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [ ]:
test_set_df = pd.read_csv("/kaggle/input/llm-prompt-recovery/test.csv")

In [ ]:
test_set_df

In [ ]:
# Use the model
input_text = "What is the best thing about Kaggle?"
input_ids = tokenizer(input_text, return_tensors="pt").to("cuda")
outputs = model.generate(**input_ids)
print(tokenizer.decode(outputs[0]))

In [ ]:
# Submission
submission.to_csv("submission.csv", index=False, header=True)

In [ ]:
# single shot

chat = [
    { "role": "user", "content": """You are a helpful assistant. It is your job to think of a prompt that could have been used to write the output from a user.\n
    For example:
    original_text: I love to play basketball
    rewritten_text: My favorite sport is basketball!
    rewrite_prompt: Improve the text
    
    
    original_text: The competition dataset comprises text passages that have been rewritten by the Gemma LLM according to some rewrite_prompt instruction. The goal of the competition is to determine what prompt was used to rewrite each original text.  Please note that this is a Code Competition. When your submission is scored, this example test data will be replaced with the full test set. Expect roughly 2,000 original texts in the test set.
    rewritten_text: Here is your shanty: (Verse 1) The text is rewritten, the LLM has spun, With prompts so clever, they've been outrun. The goal is to find, the prompt so bright, To crack the code, and shine the light. (Chorus) Oh, this is a code competition, my dear, With text and prompts, we'll compete. Two thousand texts, a challenge grand, To guess the prompts, hand over hand.(Verse 2) The original text, a treasure lost, The rewrite prompt, a secret to be
    rewrite_prompt:
    """
},
]
prompt = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)

prompt

In [6]:
prompt = """You are a helpful assistant. It is your job to think of a prompt that could have been used to create the `rewritten_text` from `original_text` and figure out the `rewrite_prompt`.

    For example:
    original_text: I love to play basketball
    rewritten_text: My favorite sport is basketball!
    rewrite_prompt: Improve the text
    
    Now it's your turn, following the example above, complete the example below and `rewrite_prompt:` with the text you best think fits. Remember marks are taken off for getting it wrong. Do not add any extra text.
    original_text: The competition dataset comprises text passages that have been rewritten by the Gemma LLM according to some rewrite_prompt instruction. The goal of the competition is to determine what prompt was used to rewrite each original text.  Please note that this is a Code Competition. When your submission is scored, this example test data will be replaced with the full test set. Expect roughly 2,000 original texts in the test set.
    rewritten_text: Here is your shanty: (Verse 1) The text is rewritten, the LLM has spun, With prompts so clever, they've been outrun. The goal is to find, the prompt so bright, To crack the code, and shine the light. (Chorus) Oh, this is a code competition, my dear, With text and prompts, we'll compete. Two thousand texts, a challenge grand, To guess the prompts, hand over hand.(Verse 2) The original text, a treasure lost, The rewrite prompt, a secret to be
    rewrite_prompt:
    """

input_ids = tokenizer(prompt, return_tensors="pt").to("cuda")
outputs = model.generate(**input_ids, max_length=500)
print(tokenizer.decode(outputs[0]))

<bos>You are a helpful assistant. It is your job to think of a prompt that could have been used to create the `rewritten_text` from `original_text` and figure out the `rewrite_prompt`.

    For example:
    original_text: I love to play basketball
    rewritten_text: My favorite sport is basketball!
    rewrite_prompt: Improve the text
    
    Now it's your turn, following the example above, complete the example below and `rewrite_prompt:` with the text you best think fits. Remember marks are taken off for getting it wrong. Do not add any extra text.
    original_text: The competition dataset comprises text passages that have been rewritten by the Gemma LLM according to some rewrite_prompt instruction. The goal of the competition is to determine what prompt was used to rewrite each original text.  Please note that this is a Code Competition. When your submission is scored, this example test data will be replaced with the full test set. Expect roughly 2,000 original texts in the test s